# Get Public Companies

In [1]:
import eikon as ek
import numpy as np
import pandas as pd

from utilities import filter_dataset

In [2]:
with open('key.secret') as file:
    KEY = file.read()

In [3]:
try:
    ek.set_app_key(KEY)
    bonds, err = ek.get_data(['CN151819SH='], 'TR.FirstAnnounceDate')
    if bonds['First Announcement Date'][0] != '2019-07-11':  # check Eikon connection
        raise ValueError('Eikon connection not successful.')
except ek.EikonError:
    raise ValueError('Please check key.secret, then run Eikon Desktop or Eikon API Proxy')

In [4]:
green_bonds = pd.read_excel('db.xlsx', parse_dates=['Issue Date'])
green_bonds = filter_dataset.filter_dataset(green_bonds)

Excluded 384 rows over 2255. Green bonds before 2009 must be due to errors and 2020 might pollute data due to COVID and year not finished yet.


In [5]:
green_bonds.columns

Index(['Issuer', 'Ticker', 'Coupon', 'Maturity', 'Issue Date', 'ISIN',
       'Preferred RIC', 'Principal Currency', 'Country of Issue',
       'Issuer Type', 'Instrument Type', 'Coupon Type', 'Amount Issued (USD)',
       'Bond Grade', 'Green Bond', 'Sector', 'TRBC Sector',
       'Country of Incorporation', 'SIC', 'Use of Proceeds'],
      dtype='object')

In [6]:
# 17 missing ISIN
# where no ISIN, no IsPublic, but we have RIC except in one case
green_bonds[green_bonds.ISIN.isna()][['ISIN', 'Preferred RIC', 'Ticker']]

,ISIN,Preferred RIC,Ticker
57,NaN,CN114265SZ=,SZGCN
90,NaN,CN143952SH=,SUNTG
139,NaN,CN150502SH=,TCLEA
226,NaN,CN150701SH=,TCLEA
253,NaN,CN155057SH=,CNLYP
297,NaN,CN155956SH=,SUNTG
300,NaN,CN143518SH=,SHMNNP
585,NaN,CN143745SH=,SASADH
592,NaN,CN143525SH=,CEWL
598,NaN,CN150646SH=,XINJGQ


In [7]:
bonds, err = ek.get_data(green_bonds.ISIN.dropna().to_list(), 
                         'TR.IsPublic')
bonds

,Instrument,Organization Is Public Flag
0,XS1890845875,False
1,CND10000C3L5,True
2,XS2082433736,False
3,CND100017CJ8,False
4,CND10001WQ91,False
...,...,...
1849,XS1207105161,False
1850,US83417KFC71,False
1851,XS1512929842,False
1852,XS1566937154,False


In [8]:
green_bonds['IsPublic'] = green_bonds.ISIN.map(dict(zip(bonds.Instrument,
                                                        bonds['Organization Is Public Flag'])))

In [9]:
green_bonds[green_bonds.IsPublic.isna()][['ISIN', 'IsPublic', 'Preferred RIC']]

,ISIN,IsPublic,Preferred RIC
57,NaN,NaN,CN114265SZ=
90,NaN,NaN,CN143952SH=
139,NaN,NaN,CN150502SH=
226,NaN,NaN,CN150701SH=
253,NaN,NaN,CN155057SH=
297,NaN,NaN,CN155956SH=
300,NaN,NaN,CN143518SH=
585,NaN,NaN,CN143745SH=
592,NaN,NaN,CN143525SH=
598,NaN,NaN,CN150646SH=


In [11]:
bonds, err = ek.get_data(green_bonds[green_bonds.IsPublic.isna()]['Preferred RIC'].dropna().to_list(), 
                         'TR.IsPublic')
bonds

,Instrument,Organization Is Public Flag
0,CN114265SZ=,False
1,CN143952SH=,True
2,CN150502SH=,False
3,CN150701SH=,False
4,CN155057SH=,True
5,CN155956SH=,True
6,CN143518SH=,False
7,CN143745SH=,False
8,CN143525SH=,True
9,CN150646SH=,False


In [12]:
rics_to_ispublic = dict(zip(bonds.Instrument, bonds['Organization Is Public Flag']))

In [18]:
green_bonds['IsPublic'] = green_bonds['IsPublic'].fillna(green_bonds['Preferred RIC'].map(rics_to_ispublic))

In [20]:
green_bonds.to_csv('green_bonds_IsPublic.csv', index=False)